# Training a LLM based on Transformer Architecture #


Importing Relevant Pytorch Libraries

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Defining  Hyperparameters for the Model

In [35]:
#Parameters
batch_size = 20
context_len = 100
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = torch.device('mps') 
eval_iters = 200
embd_dim = 384
n_head = 6
n_layer = 6
dropout = 0.2

Importing our dataset on which we will train the LLM.

In [36]:
#Load data (Enter any file name)
with open('/Users/hitengoel/Machine Learning/GPT Project/shakespear_data copy.txt', 'r', encoding='utf-8') as f:
    text = f.read()

Creating a vocabulary and encoder and decoder functions.

In [37]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

Creating a Test and Validation set.

In [38]:
#Test and Validation Split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # 90% train, 10% test
train_data = data[:n]
val_data = data[n:]

Making Batches of the training examples , in order to speed up the training process using parallel processing by GPU.

In [39]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_len, (batch_size,))
    x = torch.stack([data[i:i+context_len] for i in ix])
    y = torch.stack([data[i+1:i+context_len+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Creating a function to estimate the average loss over various batches.

In [40]:
@torch.no_grad() #this enables pytorch to skip gradient computation
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() # mean loss over the eval_iters batches
    model.train()
    return out

## "Attention is All you Need" ##

Creating one Self-Attention *Head* to be used in an Attention Block in the Transformer.

In [41]:
class Head(nn.Module):
    # One head of Self-Attention

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embd_dim, head_size, bias=False) # Creating a Linear operation to get the keys
        self.query = nn.Linear(embd_dim, head_size, bias=False) # Creating a Linear operation to get the queries
        self.value = nn.Linear(embd_dim, head_size, bias=False) # Creating a Linear operation to get the values
        self.register_buffer('tril', torch.tril(torch.ones(context_len, context_len))) # Lower triangular matrix 

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores 
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # mask out the lower triangular part of the attention matrix to prevent attending to future tokens
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) 

        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

Creating one Attention Block with multiple Heads.

In [42]:
class MultiHeadAttention(nn.Module):
    # Running multiple heads in parallel

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embd_dim) #Residual connection
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

Creating a Feed-Forward layer.

In [43]:
class FeedFoward(nn.Module):
    # Simple linear feedforward network

    def __init__(self, embd_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embd_dim, 4 * embd_dim),
            nn.ReLU(),
            nn.Linear(4 * embd_dim, embd_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

Combining one Attention block and one feed-forward layer to create one *Block* of the transformer.

In [44]:
class Block(nn.Module):
    # Tokens communicate with each other through self-attention and computation happens in feedforward network

    def __init__(self, embd_dim, n_head):
        # embd_dim: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = embd_dim // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(embd_dim)
        self.ln1 = nn.LayerNorm(embd_dim) # Layer Normalization for the self-attention output
        self.ln2 = nn.LayerNorm(embd_dim) # Layer Normalization for the feedforward output

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

Putting it All together to create the LLM.

In [45]:
class Language_Model(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_emb_table = nn.Embedding(vocab_size, embd_dim)
        self.position_emb_table = nn.Embedding(context_len, embd_dim)
        self.blocks = nn.Sequential(*[Block(embd_dim, n_head=n_head) for i in range(n_layer)]) #Adding multiple layers to the model
        self.ln_f = nn.LayerNorm(embd_dim) # Final Layer Normalization
        self.lm_head = nn.Linear(embd_dim, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_emb_table(idx) # (B,T,C)
        pos_emb = self.position_emb_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Dimensionality adjusted due to requirement of cross entropy loss function
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        else:
            loss = None

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last context_len tokens
            idx_cond = idx[:, -context_len:]
            # predictions for the next token
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Training And Saving the Model.

In [46]:
model = Language_Model()
m = model.to(device)
# printing the number of parameters in the model
print("Number of parameters in the model: ")
print(sum(p.numel() for p in m.parameters())/1e5, 'Lakh parameters')

# creating a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: training loss:{losses['train']:.4f} || validation loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Saving the model

torch.save(model.state_dict(), 'shakespeare_model.pth')
print('Model trained and saved!')

Number of parameters in the model: 
107.5594 Lakh parameters
step 0: training loss:4.6035 || validation loss: 4.6047
step 500: training loss:2.1222 || validation loss: 2.1331
step 1000: training loss:1.8605 || validation loss: 1.9070
step 1500: training loss:1.7356 || validation loss: 1.7850
step 2000: training loss:1.6433 || validation loss: 1.7245
step 2500: training loss:1.5689 || validation loss: 1.6788
step 3000: training loss:1.5219 || validation loss: 1.6346
step 3500: training loss:1.4866 || validation loss: 1.6069
step 4000: training loss:1.4574 || validation loss: 1.5965
step 4500: training loss:1.4333 || validation loss: 1.5806
step 4999: training loss:1.4068 || validation loss: 1.5646
Model trained and saved!


Generating a sample output from the model

In [47]:
new_model = Language_Model()
nm = new_model.to(device)
new_model.load_state_dict(torch.load('shakespeare_model.pth'))
new_context = torch.zeros((1, 1), dtype=torch.long, device=device)

open('Sample_text_shakespear.txt', 'w').write(decode(nm.generate(new_context, max_new_tokens=10000)[0].tolist()))
print('Sample text generated and saved!')

Sample text generated and saved!
